## Problema de Roteirização de Veículos

A roteirização de veículos trata-se de um problema comum, cuja solução é buscada por meio de técnicas de otimização.

Geralmente, problemas desse tipo envolvem a redução de movimentação, e por consequência dos custos logísticos de veículos na prestação de algum tipo de serviço que envolve a visitação de diferentes pontos.

No caso em questão, foi criada uma situação hipotética na qual produtos são distribuídos a partir da Ceanorte para diferentes consumidores. Nesse caso, para os seguintes consumidores hipotéticos:

| Local | Latitude | Longitude |
| --- | --- | --- |
| Center Pão| 	-16.729240| 	-43.875400
| Cordeiro Supermercados - João XXIII | 	-16.715024 | 	-43.863827
| Cordeiro Atacarejo Plinio Ribeiro | 	-16.741572 | 	-43.845966
| Cordeiro Atacarejo Planalto | 	-16.697158 | 	-43.841259
| Hiper Bretas Shopping | 	-16.741286 | 	-43.869367
| Bretas - Coronel Prates | 	-16.723195 | 	-43.868241
| Bretas - Mestre Janjão | 	-16.719810 | 	-43.859490
| Superação Supermercados | 	-16.748192 | 	-43.853606
| Supermercados BH - Plínio Ribeiro | 	-16.743915 | 	-43.846836
| Supermercados BH - Donato Quintino | 	-16.739069 | 	-43.870327
| Ceanorte | 	-16.756940| 	-43.845899


Uma rota deverá ser estabelecida a partir de Ceanorte e distribuir para dos demais centros consumidores, contudo, essa rota deverá ser a com menor custo logístico possível, nesse caso o custo logístico será representado pelas distâncias geográficas.

In [1]:
# Bibliotecas a serem usadas

import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
from scipy.spatial import distance_matrix
from itertools import permutations
import random
from deap import base, creator, tools, algorithms

### 1. Dados e transformações

In [2]:
# Ingestão da planilha excel com locais, latitude e longitudes.

df = pd.read_excel("distancias.xlsx")
df

,local,latitude,longitude
0,Center Pão,-16.729240,-43.875400
1,Cordeiro Supermercados - João XXIII,-16.715024,-43.863827
2,Cordeiro Atacarejo Plinio Ribeiro,-16.741572,-43.845966
3,Cordeiro Atacarejo Planalto,-16.697158,-43.841259
4,Hiper Bretas Shopping,-16.741286,-43.869367
5,Bretas - Coronel Prates,-16.723195,-43.868241
6,Bretas - Mestre Janjão,-16.719810,-43.859490
7,Superação Supermercados,-16.748192,-43.853606
8,Supermercados BH - Plínio Ribeiro,-16.743915,-43.846836
9,Supermercados BH - Donato Quintino,-16.739069,-43.870327


In [3]:
# Função que calcula as distâncias relativas entre todos os locais

def create_distance_matrix(df):
    # Inicializa um DataFrame vazio com índices e colunas do mesmo tamanho que df['local']
    distance_matrix = pd.DataFrame(index=df['local'], columns=df['local'])
    
    # Itera sobre cada par de locais para calcular a distância geodésica
    for loc1 in df.index:
        for loc2 in df.index:
            # Obtem as coordenadas de cada local
            coord1 = (df.at[loc1, 'latitude'], df.at[loc1, 'longitude'])
            coord2 = (df.at[loc2, 'latitude'], df.at[loc2, 'longitude'])
            
            # Calcula a distância e preenche na célula correspondente da matriz
            # Distâncias são arredondadas para duas casas decimais
            distance_matrix.at[df.at[loc1, 'local'], df.at[loc2, 'local']] = round(geodesic(coord1, coord2).kilometers, 2)

    return distance_matrix

In [4]:
# Chamando a função e passando o DataFrame 'df'
distance_matrix = create_distance_matrix(df)

# Mostrando o DataFrame resultante
distance_matrix

local,Center Pão,Cordeiro Supermercados - João XXIII,Cordeiro Atacarejo Plinio Ribeiro,Cordeiro Atacarejo Planalto,Hiper Bretas Shopping,Bretas - Coronel Prates,Bretas - Mestre Janjão,Superação Supermercados,Supermercados BH - Plínio Ribeiro,Supermercados BH - Donato Quintino,Ceanorte
local,,,,,,,,,,,
Center Pão,0.0,2.0,3.42,5.09,1.48,1.02,1.99,3.13,3.45,1.21,4.39
Cordeiro Supermercados - João XXIII,2.0,0.0,3.5,3.11,2.97,1.02,0.7,3.83,3.67,2.75,5.02
Cordeiro Atacarejo Plinio Ribeiro,3.42,3.5,0.0,4.94,2.5,3.13,2.81,1.1,0.28,2.61,1.7
Cordeiro Atacarejo Planalto,5.09,3.11,4.94,0.0,5.73,4.07,3.17,5.8,5.21,5.58,6.63
Hiper Bretas Shopping,1.48,2.97,2.5,5.73,0.0,2.01,2.6,1.85,2.42,0.27,3.04
Bretas - Coronel Prates,1.02,1.02,3.13,4.07,2.01,0.0,1.01,3.18,3.24,1.77,4.43
Bretas - Mestre Janjão,1.99,0.7,2.81,3.17,2.6,1.01,0.0,3.2,2.99,2.42,4.36
Superação Supermercados,3.13,3.83,1.1,5.8,1.85,3.18,3.2,0.0,0.86,2.05,1.27
Supermercados BH - Plínio Ribeiro,3.45,3.67,0.28,5.21,2.42,3.24,2.99,0.86,0.0,2.56,1.44


### 2. Problema do Caixeito Viajante 

O Problema do Caixeiro Viajante (TSP - Traveling Salesman Problem) é um problema clássico na pesquisa operacional e na ciência da computação, especialmente no campo da otimização combinatória. Ele se destaca não apenas pela complexidade em encontrar soluções ótimas para instâncias grandes, mas também pela variedade de suas aplicações em diferentes áreas.

#### 2.1 O algoritmo "Nearest Neighbour" 
O algoritmo "Nearest Neighbour" (Vizinho Mais Próximo) é uma abordagem heurística simples para resolver o problema do caixeiro viajante (TSP - Traveling Salesman Problem). O objetivo do TSP é encontrar a rota mais curta que permite a um caixeiro viajante visitar uma série de cidades uma única vez e retornar à cidade de origem. O algoritmo do Vizinho Mais Próximo é popular devido à sua facilidade de implementação e à rapidez com que produz uma solução razoavelmente boa, embora nem sempre seja a ótima.

In [5]:
def nearest_neighbour_route(distance_matrix, starting_point):
    # Inicializa a rota com o ponto de origem
    route = [starting_point]
    total_distance = 0
    
    # Cria um conjunto de locais não visitados
    to_visit = set(distance_matrix.index)
    to_visit.remove(starting_point)
    
    # Enquanto houver locais não visitados, continua o loop
    while to_visit:
        current_location = route[-1]
        # Encontra o vizinho mais próximo que ainda não foi visitado
        nearest_neighbour = min(to_visit, key=lambda x: distance_matrix.at[current_location, x])
        # Adiciona a distância ao vizinho mais próximo ao total
        total_distance += distance_matrix.at[current_location, nearest_neighbour]
        # Adiciona o vizinho mais próximo à rota
        route.append(nearest_neighbour)
        # Remove o vizinho mais próximo do conjunto de locais não visitados
        to_visit.remove(nearest_neighbour)
    
    # Retorna a rota e a distância total
    return route, total_distance

# Supondo que 'distance_matrix_df' é o DataFrame da matriz de distâncias que já foi calculada
# Chamando a função e passando a matriz de distâncias e o ponto de partida desejado
starting_point = 'Ceanorte'
route, total_distance = nearest_neighbour_route(distance_matrix, starting_point)

# Imprimindo a rota e a distância total
print("Rota de Entregas:", route)
print("Distância Total Percorrida: {:.2f} km".format(total_distance))
    

Rota de Entregas: ['Ceanorte', 'Superação Supermercados', 'Supermercados BH - Plínio Ribeiro', 'Cordeiro Atacarejo Plinio Ribeiro', 'Hiper Bretas Shopping', 'Supermercados BH - Donato Quintino', 'Center Pão', 'Bretas - Coronel Prates', 'Bretas - Mestre Janjão', 'Cordeiro Supermercados - João XXIII', 'Cordeiro Atacarejo Planalto']
Distância Total Percorrida: 12.23 km


Aqui, observa-se que o algoritmo de vizinho mais próximo determinou uma rota com custo de 12.23 Km

#### 2.2 O algoritmo genético

O algoritmo genético aplicado ao Problema do Caixeiro Viajante (TSP) é uma abordagem metaheurística inspirada nos processos de seleção natural e genética. Ele é particularmente útil no TSP devido à sua habilidade de encontrar soluções de boa qualidade em um espaço de pesquisa complexo e vasto, como é o caso das rotas possíveis no TSP.

In [6]:
origin = 10

# Função de avaliação modificada para começar no ponto de origem
def evaluate(individual):
    distance = distance_matrix.iloc[origin, individual[0]]
    for i in range(len(individual) - 1):
        distance += distance_matrix.iloc[individual[i], individual[i+1]]
    return distance,

# Modificação na criação de um indivíduo para começar sempre pelo ponto de origem
def create_individual():
    indices = list(range(len(distance_matrix)))
    indices.remove(origin)
    random.shuffle(indices)
    return [origin] + indices

# Configuração do algoritmo genético
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Parâmetros do algoritmo
population = toolbox.population(n=300)
ngen = 100
cxpb = 0.8
#mutpb = 0.2
mutpb = 0.2

# Executando o algoritmo
random.seed(64)
pop, log = algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngen, verbose=False)

# Melhor solução


best_ind = tools.selBest(pop, 1)[0]
best_route = [distance_matrix.index[i] for i in best_ind]
print("Melhor roteiro:", best_ind)
print("Melhor roteiro:", best_route)
print("Distância total:", evaluate(best_ind)[0])


Melhor roteiro: [10, 8, 2, 7, 4, 9, 0, 5, 6, 1, 3]
Melhor roteiro: ['Ceanorte', 'Supermercados BH - Plínio Ribeiro', 'Cordeiro Atacarejo Plinio Ribeiro', 'Superação Supermercados', 'Hiper Bretas Shopping', 'Supermercados BH - Donato Quintino', 'Center Pão', 'Bretas - Coronel Prates', 'Bretas - Mestre Janjão', 'Cordeiro Supermercados - João XXIII', 'Cordeiro Atacarejo Planalto']
Distância total: 11.989999999999998


#### Análise dos resultados

 - Algoritmo Nearest Neighbour: 12.23 Km
 - Algoritmo Genético: 11.98 Km

Existe uma diferença de 250m entre os roteiros, então decide-se seguir o roteiro indicado pelo algoritmo genético. Supondo que seriam feitas 3 reposições de produtos na semana, podemos considerar que, aproximadamente, existe o potencial de economizar um total 4Km por mês. Considerando veículo com autonomia de 10Km por litro de gasolina, representaria uma economia de 0,25l por mês.
Essa econommia pode parecer pequena, mas se aumentarmos a escala do problema, incluindo novos locais e veículos, logo seria visto o crescimento de custos de forma exponencial. Esses custos seriam, da mesma forma, reduzidos pela aplicação desse algoritmo.



